# Heart experiments

This is a Jupytor notebook that reproduces some of the results shown in paper "Collinear datasets augmentation using Procrustes validation sets" (in a more simple form).  

First we load packages and define ANN discrimination model for Heart experiments.

In [9]:
import random
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader, TensorDataset
from prcv.methods import pcvpls
from pandas import read_csv

# ANN learning parameters
N_EPOCH = 300    # number of epochs
LR = 0.000001    # learning rate
BATCH_SIZE = 10  # batch size

device = 'cuda' if torch.cuda.is_available() else 'cpu'

class ANNModel(nn.Module):
    def __init__(self):
        super(ANNModel, self).__init__()
        self.fc1 = nn.Linear(in_features=17, out_features=34)
        self.fc20 = nn.Linear(in_features=34, out_features=68)
        self.fc21 = nn.Linear(in_features=68, out_features=68)
        self.fc22 = nn.Linear(in_features=68, out_features=68)
        self.fc3 = nn.Linear(in_features=68, out_features=34)
        self.fc4 = nn.Linear(in_features=34, out_features=1)
        self.fc5 = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.fc20(x)
        x = nn.functional.relu(x)
        x = self.fc21(x)
        x = nn.functional.relu(x)
        x = self.fc22(x)
        x = nn.functional.relu(x)
        x = self.fc3(x)
        x = nn.functional.relu(x)
        x = self.fc4(x)
        x = self.fc5(x)
        return x

# method for making predictions (returns predicted response values, and accuracy)
def predict(model, X, Y):
    X_tensor = torch.from_numpy(X).float().to(device)
    Yp_tensor = model(X_tensor)
    Yp = Yp_tensor.detach().cpu().numpy()
    TP = (Yp[:, 0] > 0.5) == (Y[:, 0] > 0.5)
    Acc = sum(TP) / Y.shape[0]
    return Yp, Acc

# method for counting all model parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [10]:
# initialize model and show number of parameters
model = ANNModel()
count_parameters(model)

14757

Function which generates `n` PV-sets using PLS based algorithm with `ncomp` components and random splits into `nseg` segments and then train the ANN model based on the augmented data. If `n = 0` model will be trained using the original training set, without augmentation.

In [11]:
def doit(Xc, Yc, n, ncomp, nseg):

    # initially augmented training set is the same as the original one
    Xc_aug = Xc
    Yc_aug = Yc

    if (n > 0):
        # loop for generation of additional data points using PCV for PLS and adding them
        # to the training set
        cv = {'type': 'rand', 'nseg': nseg}
        for i in range(n):
            Xpv, D = pcvpls(Xc, Yc, ncomp = ncomp, center = True, scale = False, cv = cv)
            Xc_aug = np.concatenate((Xc_aug, Xpv), axis = 0)
            Yc_aug = np.concatenate((Yc_aug, Yc), axis = 0)

    # turn training set to Torch tensors
    Xc_tensor = torch.from_numpy(Xc_aug).float().to(device)
    Yc_tensor = torch.from_numpy(Yc_aug).float().to(device)

    # create Torch dataset and data loader with given batch size
    cal = TensorDataset(Xc_tensor, Yc_tensor)
    cal_loader = DataLoader(cal, batch_size = BATCH_SIZE, shuffle = True)

    # learning using given number of epochs and learning rate
    model = ANNModel().to(device)
    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr = LR)
    losses = np.zeros(N_EPOCH, dtype = np.float64)
    for epoch in range(N_EPOCH):
        for i, (inputs, targets) in enumerate(cal_loader):
            # zero the gradients
            optimizer.zero_grad()

            # forward pass
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            losses[epoch] += loss

            # backward pass and optimization
            loss.backward()
            optimizer.step()

    return model, losses

Load, split and preprocess dataset.

In [12]:
# load data from CSV file (it already contains dummy variables)
d = read_csv('Heart.csv', delimiter=",", decimal=".")
n = d.shape[0]

# split to X and Y
Y = d.iloc[:, 17:18].to_numpy()
X = d.iloc[:, 0:17].to_numpy()

# split to members and strangers
ind0 = Y[:,0] == 0
ind1 = Y[:,0] == 1
X0 = X[ind0, ]
Y0 = Y[ind0, ]
X1 = X[ind1, ]
Y1 = Y[ind1, ]

n0 = X0.shape[0]
n1 = X1.shape[0]

# compute size of calibration and test set based on 75/25 split
n0_cal = round(n0 * 0.75)
n0_tst = n0 - n0_cal
n1_cal = round(n1 * 0.75)
n0_tst = n1 - n1_cal

Now we repeat the training/test procedure 10 times, save all Accuracies (this will take some time).

In [13]:
n = 10
ACC1_v = np.zeros(n)
ACC2_v = np.zeros(n)

for i in range(n):

    # split data to training set and calibration set in each class
    ind0_all = list(range(n0))
    random.shuffle(ind0_all)
    ind0_cal = ind0_all[0:n0_cal]
    ind0_tst = ind0_all[n0_cal:n0]

    ind1_all = list(range(n1))
    random.shuffle(ind1_all)
    ind1_cal = ind1_all[0:n1_cal]
    ind1_tst = ind1_all[n1_cal:n1]

    X0c = X0[ind0_cal, ]
    Y0c = Y0[ind0_cal, ]
    X0t = X0[ind0_tst, ]
    Y0t = Y0[ind0_tst, ]

    X1c = X1[ind1_cal, ]
    Y1c = Y1[ind1_cal, ]
    X1t = X1[ind1_tst, ]
    Y1t = Y1[ind1_tst, ]

    # combine the classes together to form calibration set
    Xc = np.vstack((X0c, X1c))
    Yc = np.vstack((Y0c, Y1c))

    # combine the classes together to form test set
    Xt = np.vstack((X0t, X1t))
    Yt = np.vstack((Y0t, Y1t))

    # compute mean and std for the calibration set
    mXc = Xc.mean(axis=0)
    sXc = Xc.std(axis=0)

    # autoscale both sets using the computed values
    Xc = (Xc - mXc) / sXc
    Xt = (Xt - mXc) / sXc

    # train the models. Model 1 is based on original training set. Model 2 is based on training
    # set augmented with 10 PV-sets computed using 10 latent variables and cross-validation
    # splits with 10 segments.
    model1, losses1 = doit(Xc, Yc,  0,  0,  0)
    model2, losses2 = doit(Xc, Yc, 10, 10, 10)

    # make predictions for the test set
    Yp1, ACC1_v[i] = predict(model1, Xt, Yt)
    Yp2, ACC2_v[i] = predict(model2, Xt, Yt)
    print(i, ACC1_v[i], ACC2_v[i])

0 0.4520547945205479 0.863013698630137


Compare the results visually.

In [ ]:
p = plt.boxplot(np.column_stack((ACC1_v, ACC2_v)), labels = ["No augmentation", "Augmented"])